In [1]:
import os
import json
import random
import traceback

from datetime import datetime

from tqdm.notebook import tqdm

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.metrics.fact_score.fact_scorer import FactScorer
from experiments.metrics.bert_score import BertScore
from experiments.metrics.llm_judge import LLMAsJudgeEvaluator

In [2]:
USE_MCP = False
DEBUG_MODE = False
USE_TRIAGE = False

LLM_JUDGE_MODEL = "anthropic--claude-3.5-sonnet"

fact_scorer = FactScorer()
path_to_source_data = "./resources/02_facts/"

date_str = datetime.now().strftime("%d%m%Y")
path_to_results = f"./results/{date_str}"

if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

time_str = datetime.now().strftime("%H%M%S")

result_file_name = f"{path_to_results}/{time_str}_results.json"
failed_experiments_file_name = f"{path_to_results}/{time_str}_failed_experiments.json"

In [3]:
file_path = os.path.abspath(path_to_source_data + "data_set_facts.json")

with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
data_set_for_experiment = data_set[:10]

In [5]:
triage_agent = Triage()
execution_trail = ""
experiment_results: list[ExperimentResult] = []
failed_experiments: list = []

for data_row in tqdm(data_set_for_experiment, desc="Running cases", unit="case"):
    try:
        result = ExperimentResult(
            id=data_row.id,
            question=data_row.question,
            answer=data_row.answer,
            product=data_row.product,
            category=data_row.category,
            persona=data_row.persona,
            activity=data_row.activity,
            country=data_row.country,
        )
        print(f"Asking agent question with id: {data_row.id}")
        if USE_TRIAGE:
            print("\tUsing Triage to categorize the query...")
            query_categorization = triage_agent.triage_user_message(
                user_message=data_row.question
            )
            if DEBUG_MODE:
                print(f"\tQuery categorization: {query_categorization}")

        result.triage_category = (
            query_categorization["category"]
            if USE_TRIAGE
            else TriageSettings.Categories.ALL
        )

        tools = ToolsFabric.get_tools_for_category(
            use_mcp=USE_MCP,
            configuration=result.triage_category,
        )

        if USE_MCP:
            async with MultiServerMCPClient(tools) as client:
                agent = ReActAgent(tool_list=client.get_tools())
                execution_trail = await agent.arun_agent_with_input(
                    user_message=data_row.question, debug=DEBUG_MODE
                )
        else:
            agent = ReActAgent(tool_list=tools)
            execution_trail = agent.run_agent_with_input(
                user_message=data_row.question, debug=DEBUG_MODE
            )

        run_data = agent.get_execution_data()

        result.tools_used = run_data.tools_used
        result.tool_calls_count = len(run_data.tools_used)
        result.excecution_time_seconds = run_data.excecution_time_seconds
        result.model_used = run_data.model_used
        result.tokens_consumed = run_data.tokens_consumed
        result.llm_call_count = run_data.llm_call_count
        result.facts = data_row.facts
        result.generated_answer = run_data.final_output

        print("\tFinished agent execution")

        print("\tCalculating FactScore...")
        result.fact_score = await fact_scorer.get_fact_score(
            facts=data_row.facts,
            knowledge_source=result.generated_answer,
            debug=DEBUG_MODE,
        )

        print("\tCalculating BERTScore...")
        result.bert_score = BertScore.compute_score(
            expected_response=data_row.answer, actual_response=result.generated_answer
        )

        print("\tEvaluating agent response with LLM as a Judge...")
        result.llm_judge_model = LLM_JUDGE_MODEL
        llm_evaluator = LLMAsJudgeEvaluator(model=LLM_JUDGE_MODEL)

        result.llm_judge_outcome = llm_evaluator.evaluate(
            question=data_row.question, generated_answer=result.generated_answer
        )

        result.llm_judge_call_count = llm_evaluator.get_llm_call_count()
        result.llm_judge_tokens_consumed = llm_evaluator.get_token_consumption()

        print(f"\tFinished experiment for id: {data_row.id}\n")

        experiment_results.append(result)
    except Exception as e:
        print(f"\tExperiment failed for id {data_row.id}: {e}")
        if DEBUG_MODE:
            traceback.print_exc()

        failed_experiments.append(
            {
                "id": data_row.id,
                "error": str(e),
                "traceback": traceback.format_exc(),
            }
        )

Running cases:   0%|          | 0/10 [00:00<?, ?case/s]

Asking agent question with id: EA-1
	Finished agent execution
	Calculating FactScore...
	Experiment failed for id EA-1: Error while parsing the response of the LLM
Asking agent question with id: EA-2
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...


/Users/i589287/Documents/Repositories/master_thesis/react_agent/src/util/llm_proxy.py:93: RuntimeWarning: coroutine 'AsyncSession.async_client' was never awaited
  self._llm = init_llm(


	Finished experiment for id: EA-2

Asking agent question with id: EA-3
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...


/Users/i589287/Documents/Repositories/master_thesis/react_agent/src/util/llm_proxy.py:93: RuntimeWarning: coroutine 'AsyncSession.async_client' was never awaited
  self._llm = init_llm(


...

/Users/i589287/Documents/Repositories/master_thesis/react_agent/src/util/llm_proxy.py:93: RuntimeWarning: coroutine 'AsyncSession.async_client' was never awaited
  self._llm = init_llm(


	Finished experiment for id: EA-8

Asking agent question with id: EA-9
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...


/Users/i589287/Documents/Repositories/master_thesis/react_agent/src/util/llm_proxy.py:93: RuntimeWarning: coroutine 'AsyncSession.async_client' was never awaited
  self._llm = init_llm(


	Finished experiment for id: EA-9

Asking agent question with id: EA-10
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...
	Finished experiment for id: EA-10



/Users/i589287/Documents/Repositories/master_thesis/react_agent/src/util/llm_proxy.py:93: RuntimeWarning: coroutine 'AsyncSession.async_client' was never awaited
  self._llm = init_llm(


In [6]:
if failed_experiments:
    with open(failed_experiments_file_name, "w") as f:
        json.dump(failed_experiments, f, indent=2)

In [7]:
# Serialize experiment results to list of dicts
import pandas as pd

records = [r.model_dump(mode="json") for r in experiment_results]
# Make it a dataframe in order to flatten nested structures
df = pd.json_normalize(records)

df

/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...e eDocument Cockpit.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...elongs to this BAdI.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(dire

,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,llm_judge_tokens_consumed.input_tokens,llm_judge_tokens_consumed.output_tokens,llm_judge_tokens_consumed.total_tokens,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,EA-2,"For a Greece customer in Public Cloud, what ar...",To enhance the generated XML file for a Greece...,ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,GR,[{'fact': 'To enhance the generated XML for a ...,0.252963,...,0.333333,0.555556,[{'fact': 'To enhance the generated XML for a ...,[{'fact': 'You can optionally create a BAdI fi...,1172,5,1177,13512,2436,15948
1,EA-3,For a Greece customer invoice process in Publi...,"Yes, you can enhance or extend the generated X...",ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,GR,[{'fact': 'You can enhance or extend the gener...,0.271650,...,0.000000,0.333333,[{'fact': 'You can enhance or extend the gener...,[{'fact': 'The method SET_OUTPUT_DATA belongs ...,940,5,945,9052,2087,11139
2,EA-4,"As a Public Cloud customer in Spain, can I ext...","Yes, as a Public Cloud customer in Spain, you ...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,ES,[{'fact': 'A Public Cloud customer in Spain ca...,0.234775,...,0.500000,0.500000,[{'fact': 'A Public Cloud customer in Spain ca...,[{'fact': 'It is possible to define that for s...,996,5,1001,5076,738,5814
3,EA-5,"As a Public Cloud customer in Italy, can I ext...","Yes, as a Public Cloud customer in Italy, you...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,IT,[{'fact': 'As a Public Cloud customer in Italy...,0.148353,...,0.333333,0.400000,[{'fact': 'As a Public Cloud customer in Italy...,[{'fact': 'You can define that for some types ...,1059,5,1064,14448,2305,16753
4,EA-6,"As a Public Cloud customer in Turkey, can I ex...","No. In Public Cloud, eDocument Actions can not...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,TR,"[{'fact': 'In Public Cloud, eDocument Actions ...",0.189163,...,0.000000,0.000000,"[{'fact': 'In Public Cloud, eDocument Actions ...",[],1063,5,1068,22663,2638,25301
5,EA-7,Can I extend an Action in eDocument Cockpit fo...,"No, Actions can not be extended in Public Cloud",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,Cross,[{'fact': 'Actions cannot be extended in Publi...,0.054198,...,0.000000,0.000000,[{'fact': 'Actions cannot be extended in Publi...,[],1095,5,1100,11350,2185,13535
6,EA-8,Can I extend an Action in eDocument Cockpit fo...,"Yes, you can extend an action in the eDocument...",ERP (excluding SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,Cross,[{'fact': 'You can extend an action in the eDo...,0.171347,...,0.000000,0.166667,[{'fact': 'You can extend an action in the eDo...,[{'fact': 'The function module is executed imm...,946,5,951,10918,2162,13080
7,EA-9,Can I extend the Manage Electronic Documents a...,"Yes, you can extend the Manage Electronic Docu...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,Cross,[{'fact': 'You can extend the Manage Electroni...,0.468703,...,1.000000,1.000000,[{'fact': 'You can extend the Manage Electroni...,[{'fact': 'Extension is possible by using Busi...,1012,5,1017,9167,1

In [8]:
# Convert to list of dictionaries (JSON objects)
json_list = df.to_dict(orient="records")

with open(result_file_name, "w") as f:
    json.dump(json_list, f, indent=2)